In [1]:
import os
import cv2
import numpy as np

In [9]:
data_directory = 'double_mnist_seed_123_image_size_64_64/test'  # Replace with the path to your data folder
class_labels = os.listdir(data_directory)

test_images = []
test_labels = []

for label in class_labels:
    label_directory = os.path.join(data_directory, label)
    for image_file in os.listdir(label_directory):
        image_path = os.path.join(label_directory, image_file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load the image in grayscale
        image = cv2.resize(image, (28, 28))  # Resize the image to a consistent size
        test_images.append(image)
        test_labels.append(int(label))  # Assuming the folder names are class labels

# Convert images and labels to NumPy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)


In [10]:
data_directory = 'double_mnist_seed_123_image_size_64_64/train'  # Replace with the path to your data folder
class_labels = os.listdir(data_directory)

train_images = []
train_labels = []

for label in class_labels:
    label_directory = os.path.join(data_directory, label)
    for image_file in os.listdir(label_directory):
        image_path = os.path.join(label_directory, image_file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load the image in grayscale
        image = cv2.resize(image, (28, 28))  # Resize the image to a consistent size
        train_images.append(image)
        train_labels.append(int(label))  # Assuming the folder names are class labels

# Convert images and labels to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [11]:
data_directory = 'double_mnist_seed_123_image_size_64_64/val'  # Replace with the path to your data folder
class_labels = os.listdir(data_directory)

val_images = []
val_labels = []

for label in class_labels:
    label_directory = os.path.join(data_directory, label)
    for image_file in os.listdir(label_directory):
        image_path = os.path.join(label_directory, image_file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load the image in grayscale
        image = cv2.resize(image, (28, 28))  # Resize the image to a consistent size
        val_images.append(image)
        val_labels.append(int(label))  # Assuming the folder names are class labels

# Convert images and labels to NumPy arrays
val_images = np.array(val_images)
val_labels = np.array(val_labels)

In [12]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert your NumPy arrays to PyTorch tensors
train_images = torch.from_numpy(train_images).float()
train_labels = torch.from_numpy(train_labels).long()
test_images = torch.from_numpy(test_images).float()
test_labels = torch.from_numpy(test_labels).long()
val_images = torch.from_numpy(val_images).float()
val_labels = torch.from_numpy(val_labels).long()

# Create TensorDatasets for training, testing, and validation
train_dataset = TensorDataset(train_images, train_labels)
test_dataset = TensorDataset(test_images, test_labels)
val_dataset = TensorDataset(val_images, val_labels)

# Define batch size
batch_size = 32  # Adjust as needed

# Create DataLoader instances for training, testing, and validation
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

# Define your MLPModel as shown below
class MLPModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc_digit1 = nn.Linear(64, num_classes)
        self.fc_digit2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        pred_digit1 = self.fc_digit1(x)
        pred_digit2 = self.fc_digit2(x)
        return pred_digit1, pred_digit2

# Hyperparameters
input_size = 28 * 28  # Input size for each digit
num_classes = 5  # Number of classes for each digit
learning_rate = 0.001
num_epochs = 10
batch_size = 32

model = MLPModel(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_pred = 0
    total_samples = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        pred_digit1, pred_digit2 = model(inputs)
        labels_digit1 = labels % 10  # Extract the first digit
        labels_digit2 = labels // 10  # Extract the second digit

        loss_digit1 = criterion(pred_digit1, labels_digit1)
        loss_digit2 = criterion(pred_digit2, labels_digit2)
        loss = loss_digit1 + loss_digit2
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted_digit1 = torch.max(pred_digit1, 1)
        _, predicted_digit2 = torch.max(pred_digit2, 1)
        correct_pred += ((predicted_digit1 == labels_digit1) & (predicted_digit2 == labels_digit2)).sum().item()        
        total_samples += labels.size(0)

    accuracy = correct_pred / total_samples
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}")

# Validation
model.eval()
val_correct_pred = 0
val_total_samples = 0

for inputs, labels in val_loader:
    pred_digit1, pred_digit2 = model(inputs)
    labels_digit1 = labels % 10  # Extract the first digit
    labels_digit2 = labels // 10  # Extract the second digit
    _, predicted_digit1 = torch.max(pred_digit1, 1)
    _, predicted_digit2 = torch.max(pred_digit2, 1)
    val_correct_pred += ((predicted_digit1 == labels_digit1) & (predicted_digit2 == labels_digit2)).sum().item()
    val_total_samples += labels.size(0)

val_accuracy = val_correct_pred / val_total_samples
print(f"Validation Accuracy: {val_accuracy:.4f}")

model.eval()
test_correct_pred = 0
test_total_samples = 0

for inputs, labels in test_loader:
    pred_digit1, pred_digit2 = model(inputs)
    labels_digit1 = labels % 10  # Extract the first digit
    labels_digit2 = labels // 10  # Extract the second digit
    _, predicted_digit1 = torch.max(pred_digit1, 1)
    _, predicted_digit2 = torch.max(pred_digit2, 1)
    test_correct_pred += ((predicted_digit1 == labels_digit1) & (predicted_digit2 == labels_digit2)).sum().item()
    test_total_samples += labels.size(0)

test_accuracy = test_correct_pred / test_total_samples
print(f"Test Accuracy: {test_accuracy:.4f}")

# You can save the trained model, perform further evaluations, and make predictions.


Epoch 1/10, Loss: 6600.8860, Accuracy: 0.1825
Epoch 2/10, Loss: 3606.6571, Accuracy: 0.4748
Epoch 3/10, Loss: 2774.1751, Accuracy: 0.5875
Epoch 4/10, Loss: 2383.1199, Accuracy: 0.6427
Epoch 5/10, Loss: 2134.8027, Accuracy: 0.6797
Epoch 6/10, Loss: 1929.7690, Accuracy: 0.7098
Epoch 7/10, Loss: 1804.7212, Accuracy: 0.7273
Epoch 8/10, Loss: 1688.7565, Accuracy: 0.7453
Epoch 9/10, Loss: 1597.7484, Accuracy: 0.7579
Epoch 10/10, Loss: 1510.9235, Accuracy: 0.7691
Validation Accuracy: 0.2684
Test Accuracy: 0.2803


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import MNIST

import torch.nn as nn

class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc_digit1 = nn.Linear(128, num_classes)
        self.fc_digit2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        pred_digit1 = self.fc_digit1(x)
        pred_digit2 = self.fc_digit2(x)
        return pred_digit1, pred_digit2


num_classes = 10
learning_rate = 0.001
num_epochs = 5
batch_size = 32

model = CNNModel(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_pred = 0
    total_samples = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs=inputs.unsqueeze(1)
        pred_digit1, pred_digit2 = model(inputs)
        labels_digit1 = labels % 10
        labels_digit2 = labels // 10

        loss_digit1 = criterion(pred_digit1, labels_digit1)
        loss_digit2 = criterion(pred_digit2, labels_digit2)
        loss = loss_digit1 + loss_digit2
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted_digit1 = torch.max(pred_digit1, 1)
        _, predicted_digit2 = torch.max(pred_digit2, 1)
        correct_pred += ((predicted_digit1 == labels_digit1) & (predicted_digit2 == labels_digit2)).sum().item()
        total_samples += labels.size(0)

    accuracy = correct_pred / total_samples
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}")

model.eval()
val_correct_pred = 0
val_total_samples = 0

for inputs, labels in val_loader:
    inputs=inputs.unsqueeze(1)
    pred_digit1, pred_digit2 = model(inputs)
    labels_digit1 = labels % 10
    labels_digit2 = labels // 10
    _, predicted_digit1 = torch.max(pred_digit1, 1)
    _, predicted_digit2 = torch.max(pred_digit2, 1)
    val_correct_pred += ((predicted_digit1 == labels_digit1) & (predicted_digit2 == labels_digit2)).sum().item()
    val_total_samples += labels.size(0)

val_accuracy = val_correct_pred / val_total_samples
print(f"Validation Accuracy: {val_accuracy:.4f}")

model.eval()
test_correct_pred = 0
test_total_samples = 0

for inputs, labels in test_loader:
    inputs=inputs.unsqueeze(1)
    pred_digit1, pred_digit2 = model(inputs)
    labels_digit1 = labels % 10
    labels_digit2 = labels // 10
    _, predicted_digit1 = torch.max(pred_digit1, 1)
    _, predicted_digit2 = torch.max(pred_digit2, 1)
    test_correct_pred += ((predicted_digit1 == labels_digit1) & (predicted_digit2 == labels_digit2)).sum().item()
    test_total_samples += labels.size(0)

test_accuracy = test_correct_pred / test_total_samples
print(f"Test Accuracy: {test_accuracy:.4f}")



Epoch 1/5, Loss: 2045.7392, Accuracy: 0.7252
Epoch 2/5, Loss: 676.0141, Accuracy: 0.8956
Epoch 3/5, Loss: 525.9506, Accuracy: 0.9176
Epoch 4/5, Loss: 444.7278, Accuracy: 0.9302
Epoch 5/5, Loss: 378.2559, Accuracy: 0.9395
Validation Accuracy: 0.6457
Test Accuracy: 0.6027


In [6]:
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset

data = np.load("permuted_mnist.npz")
permuted_x_train = data["train_images"]
y_train = data["train_labels"]
permuted_x_test = data["test_images"]
y_test = data["test_labels"]

permuted_x_train, permuted_x_val, y_train, y_val = train_test_split(permuted_x_train, y_train, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors with appropriate data types
permuted_x_train = torch.from_numpy(permuted_x_train).float()
y_train = torch.from_numpy(y_train).long()
permuted_x_val = torch.from_numpy(permuted_x_val).float()
y_val = torch.from_numpy(y_val).long()
permuted_x_test = torch.from_numpy(permuted_x_test).float()
y_test = torch.from_numpy(y_test).long()

# Create TensorDatasets
train_dataset = TensorDataset(permuted_x_train, y_train)
val_dataset = TensorDataset(permuted_x_val, y_val)
test_dataset = TensorDataset(permuted_x_test, y_test)

batch_size = 32

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define a simple MLP model
class MLP_permuted_Model(nn.Module):
    def __init__(self):
        super(MLP_permuted_Model, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)  # 10 for each digit

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = MLP_permuted_Model()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total = 0
    correct = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Calculate accuracy for this batch
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = correct / total

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = correct / total
print(f'Validation Accuracy: {val_accuracy:.4f}')

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch 1/5, Loss: 0.6480, Train Accuracy: 0.9090
Epoch 2/5, Loss: 0.3017, Train Accuracy: 0.9521
Epoch 3/5, Loss: 0.2535, Train Accuracy: 0.9590
Epoch 4/5, Loss: 0.1456, Train Accuracy: 0.9637
Epoch 5/5, Loss: 0.3409, Train Accuracy: 0.9666
Validation Accuracy: 0.9577
Test Accuracy: 0.9627


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define a simple CNN model
class CNN_permuted_Model(nn.Module):
    def __init__(self):
        super(CNN_permuted_Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 12 * 12, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 32 * 12 * 12)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
model = CNN_permuted_Model()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total = 0
    correct = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs = inputs.unsqueeze(1)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Calculate accuracy for this batch
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate and print accuracy for the entire training dataset
    train_accuracy = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.unsqueeze(1)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = correct / total
print(f'Validation Accuracy: {val_accuracy:.4f}')

# Evaluation on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.unsqueeze(1)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch 1/10, Loss: 0.2406, Training Accuracy: 0.8565
Epoch 2/10, Loss: 0.1720, Training Accuracy: 0.9452
Epoch 3/10, Loss: 0.0413, Training Accuracy: 0.9591
Epoch 4/10, Loss: 0.0390, Training Accuracy: 0.9656
Epoch 5/10, Loss: 0.1148, Training Accuracy: 0.9676
Epoch 6/10, Loss: 0.0054, Training Accuracy: 0.9714
Epoch 7/10, Loss: 0.0160, Training Accuracy: 0.9744
Epoch 8/10, Loss: 0.3376, Training Accuracy: 0.9768


KeyboardInterrupt: 